## 라이브러리 불러오기

In [4]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

# 코드 스케줄링
import threading
# import schedule

# XML 파싱
import xml.etree.ElementTree as ET

# 입력데이터 데이터프레임 작성
import pandas as pd
import numpy as np

# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60

In [2]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## 입력데이터

In [3]:
csvInputFilePath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/BusInfo/TransitBusLinePass.csv'

In [4]:
inputDataFrame = pd.read_csv(csvInputFilePath, encoding='utf-8')
inputDataFrame.shape

(1935, 7)

In [5]:
routeNoList = list(inputDataFrame["노선"])
startStationList = list(inputDataFrame["시점"])
endStationList = list(inputDataFrame["종점"])

## KEY unquote

In [46]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
# apiKEY = requests.utils.unquote(KEY)
apiKey = unquote(KEY)
# print(apiKey)
KEYSW = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
apiKeySW = unquote(KEYSW)

######
i = 1#
######

## URL 생성

In [47]:
# 노선번호목록조회 busrouteservice
# 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice' 
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('keyword') : routeNoList[i] })

## API 호출

In [48]:
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
oneLineXML = urlopen(request).read().decode('utf8')
print(oneLineXML)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader><errMsg>NORMAL SERVICE.</errMsg><returnCode>00</returnCode></comMsgHeader><msgHeader><queryTime>2020-11-11 23:42:23.038</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busRouteList><districtCd>2</districtCd><regionName>수원</regionName><routeId>200000070</routeId><routeName>11-1</routeName><routeTypeCd>13</routeTypeCd><routeTypeName>일반형시내버스</routeTypeName></busRouteList><busRouteList><districtCd>2</districtCd><regionName>광명,서울</regionName><routeId>213000007</routeId><routeName>11-1</routeName><routeTypeCd>13</routeTypeCd><routeTypeName>일반형시내버스</routeTypeName></busRouteList><busRouteList><districtCd>2</districtCd><regionName>포천</regionName><routeId>236000211</routeId><routeName>11-1</routeName><routeTypeCd>13</routeTypeCd><routeTypeName>일반형시내버스</routeTypeName></busRouteList><busRouteList><districtCd>2</districtCd><regionName>파주</regionName><routeId>2290000

In [49]:
xtree = ET.fromstring(oneLineXML)
print(len(xtree))

3


In [50]:
print(startStationList[i], endStationList[i])

busRouteList = xtree[2]
for index in range(len(busRouteList)):
    print(busRouteList[index])

곡반정동차고지 수원역환승센터(10승강장)
<Element 'busRouteList' at 0x000001D6443F5AE8>
<Element 'busRouteList' at 0x000001D644A5C8B8>
<Element 'busRouteList' at 0x000001D644A5CEF8>
<Element 'busRouteList' at 0x000001D644A63188>
<Element 'busRouteList' at 0x000001D644A633B8>
<Element 'busRouteList' at 0x000001D644A635E8>


In [51]:
for busRoute in busRouteList:
    routeId = busRoute.find("routeId").text
    
    # 노선정보항목조회 busrouteservice/info
    # 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
    url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
    queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
    
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    busInfoXML = urlopen(request).read().decode('utf8')
    
    busInfoTree = ET.fromstring(busInfoXML)
    busRouteInfoItem = busInfoTree[2]
    
    startStationName = busRouteInfoItem[0].find("startStationName").text
    endStationName = busRouteInfoItem[0].find("endStationName").text
    
    print(startStationName, endStationName)

    if(startStationList[i] == startStationName or endStationList[i] == endStationName):
        print(routeId)
        break;

곡반정동차고지 수원역환승센터(10승강장)
200000070


## 버스노선정보 API 태그 리스트

response
    comMsgHeader
    msgHeader
    msgBody
        busRouteInfoItem
            0 : "companyId" : 운수업체아이디
            1 : "companyName" : 운수업체명
            2 : "companyTel" : 운수업체 전화번호
            3 : "districtCd" : 관할지역 (1:서울,2:경기,3:인천)
            4 : "downFirstTime" : 평일 종점 첫차시간
            5 : "downLastTime" : 평일 종점 막차시간
            6 : "endMobileNo" : 종점정류소번호
            7 : "endStationId" : 종점정류소아이디
            8 : "endStationName" : 종점정류소명
            9 : "peekAlloc" : 평일 최소 배차시간
            10: "regionName" : 지역명
            11: "routeId" : 노선아이디
            12: "routeName" : 노선번호
            13: "routeTypeCd" : 노선유형
            14: "routeTypeName" : 노선유형명
            15: "startMobileNo" : 기점정류소번호
            16: "startStationId" : 기점정류소아이디
            17: "startStationName" : 기점정류소명
            18: "upFirstTime" : 평일 기점 첫차시간
            19: "upLastTime" : 평일 기점 막차시간
            20: "nPeekAlloc" : 평일 최대 배차시간
        busRouteInfoItem
    msgBody

In [69]:
# Initialize List
companyIdList = []
companyNameList = []
companyTelList = []
districtCdList = []
downFirstTimeList = []
downLastTimeList = []
endMobileNoList = []
endStationIdList = []
endStationNameList = []
peekAllocList = []
regionNameList = []
routeIdList = []
routeNameList = []
routeTypeCdList = []
routeTypeNameList = []
startMobileNoList = []
startStationIdList = []
startStationNameList = []
upFirstTimeList = []
upLastTimeList = []
nPeekAllocList = []

In [70]:
listList = \
[companyIdList,
companyNameList,
companyTelList,
districtCdList,
downFirstTimeList,
downLastTimeList,
endMobileNoList,
endStationIdList,
endStationNameList,
peekAllocList,
regionNameList,
routeIdList,
routeNameList,
routeTypeCdList,
routeTypeNameList,
startMobileNoList,
startStationIdList,
startStationNameList,
upFirstTimeList,
upLastTimeList,
nPeekAllocList]
print(listList)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [65]:
tagNameList = \
["companyId",
"companyName",
"companyTel",
"districtCd",
"downFirstTime",
"downLastTime",
"endMobileNo",
"endStationId",
"endStationName",
"peekAlloc",
"regionName",
"routeId",
"routeName",
"routeTypeCd",
"routeTypeName",
"startMobileNo",
"startStationId",
"startStationName",
"upFirstTime",
"upLastTime",
"nPeekAlloc"]
# print(tagList)

In [66]:
len(inputDataFrame)

1935

## 최종 코드

In [72]:
# for i in range(len(inputDataFrame)):
    print(i, end='\t')
    print(inputDataFrame['노선'][i], end='\t')
    flagVar = 1
    # 노선번호목록조회 busrouteservice
    # 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
    url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice' 
    queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('keyword') : routeNoList[i] })
#     print(url+queryParams)

    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    oneLineXML = urlopen(request).read().decode('utf8')
#     print(oneLineXML)

    xtree = ET.fromstring(oneLineXML)
#     print(len(xtree), end='\t')
    
    # print(startStationList[i], endStationList[i])
    resultCode = int(xtree[1].find("resultCode").text)
#     print(resultCode)
    
    if(resultCode):
        for listElement, tagName in zip(listList, tagNameList):
            listElement.append(0)
        print('0')
        continue

    busRouteList = xtree[2]
#     print(len(busRouteList))

    for busRoute in busRouteList:
        routeId = busRoute.find("routeId").text

        # 노선정보항목조회 busrouteservice/info
        # 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
#         print(url+queryParams)

        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        busInfoXML = urlopen(request).read().decode('utf8')

        busInfoTree = ET.fromstring(busInfoXML)
#         print(len(busInfoTree))
        
        busRouteInfoItem = busInfoTree[2]

        startStationName = busRouteInfoItem[0].find("startStationName").text
        endStationName = busRouteInfoItem[0].find("endStationName").text

    #     print(startStationName, endStationName)

        if(startStationList[i] == startStationName or endStationList[i] == endStationName):
#             print(routeId)
            for listElement, tagName in zip(listList, tagNameList):
                tagValue = busRouteInfoItem[0].find(tagName)
                if(tagValue == None):
                    listElement.append(None)
                    continue
                tagTextValue = tagValue.text
                listElement.append(tagTextValue)

            print(routeId)
            flagVar = 0
            break;
            
    if(flagVar):
        for listElement, tagName in zip(listList, tagNameList):
            listElement.append(-1)
        print('-1')

0	11	200000060
1	11-1	200000070
2	112	200000049
3	123A	200000269
4	123B	0
5	123C	0
6	123D	200000277
7	13	200000037
8	13-1	200000069
9	13-4	200000185
10	13-5	200000090
11	15-1	200000152
12	16	200000032
13	16-1	200000051
14	18	200000211
15	19	200000266
16	2-1	200000097
17	2-2	200000157
18	20	200000053
19	20-1	200000182
20	2007	200000120
21	22	-1
22	22-2	-1
23	25	200000055
24	25-1	-1
25	25-2	200000056
26	25-5	200000234
27	27	200000074
28	3	200000145
29	30	200000265
30	30-1	200000265
31	300	200000006
32	300-1	200000031
33	3000	200000104
34	3002	200000145
35	3003	200000108
36	3007	200000110
37	3008	-1
38	301	200000028
39	310	200000024
40	32	-1
41	32-1	-1
42	32-2	-1
43	32-3	200000236
44	32-4	200000272
45	32-5	200000134
46	34	233000011
47	34-1	233000012
48	35	200000148
49	37	-1
50	3900전	0
51	4-1	200000101
52	400	200000008
53	400-2	0
54	400-4	-1
55	400-5	0
56	42	233000087
57	46	-1
58	5	200000076
59	5-1	-1
60	5-2	-1
61	5-4	-1
62	50-2	200000016
63	50-3	200000014
64	50-4	200000018
65	50-5	2000000

In [74]:
def printList(listList):
    for listElement in range(len(listList)):
        print(len(listList[listElement]))

In [77]:
printList(listList)

1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935
1935


In [76]:
copyListList = listList

### index i 를 바꿔서 개별 노선에 대해 위 알고리즘을 실행합니다.

In [ ]:
##########
i = 1641
##########

flagVar = 1
print(i, end='\t')
# 노선번호목록조회 busrouteservice
# 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice' 
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('keyword') : routeNoList[i] })
print(url+queryParams)

request = Request(url + queryParams)
request.get_method = lambda: 'GET'
oneLineXML = urlopen(request).read().decode('utf8')
#     print(oneLineXML)

xtree = ET.fromstring(oneLineXML)
#     print(len(xtree), end='\t')

# print(startStationList[i], endStationList[i])
resultCode = int(xtree[1].find("resultCode").text)
#     print(resultCode)

if(resultCode):
    for listElement, tagName in zip(listList, tagNameList):
        listElement.append(0)
    print('0')

busRouteList = xtree[2]
#     print(len(busRouteList))

for busRoute in busRouteList:
    routeId = busRoute.find("routeId").text

    # 노선정보항목조회 busrouteservice/info
    # 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
    url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
    queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
#         print(url+queryParams)

    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    busInfoXML = urlopen(request).read().decode('utf8')

    busInfoTree = ET.fromstring(busInfoXML)
#         print(len(busInfoTree))

    busRouteInfoItem = busInfoTree[2]

    startStationName = busRouteInfoItem[0].find("startStationName").text
    endStationName = busRouteInfoItem[0].find("endStationName").text

#     print(startStationName, endStationName)

    if(startStationList[i] == startStationName or endStationList[i] == endStationName):
#             print(routeId)
        for listElement, tagName in zip(listList, tagNameList):
            tagValue = busRouteInfoItem[0].find(tagName)
            if(tagValue == None):
                listElement.append(None)
                continue
            tagTextValue = tagValue.text
            listElement.append(tagTextValue)

        print(routeId)
        flagVar = 0
        break;

if(flagVar):
    for listElement, tagName in zip(listList, tagNameList):
        listElement.append(-1)
    print('-1')

## DataFrame에 값 추가하기

In [197]:
for tagName, listElement in zip(tagNameList, listList):
    inputDataFrame.loc[:, tagName] = listElement

In [88]:
inputDataFrame.to_csv('C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/BusInfo/_FINAL.csv', encoding='utf-8-sig')

## DataFrame 형식 수정하기

In [199]:
inputDataFrame.dtypes

지역                  object
노선                  object
시점                  object
종점                  object
승차                   int64
하차                  object
전체                   int64
companyId           object
companyName         object
companyTel          object
districtCd          object
downFirstTime       object
downLastTime        object
endMobileNo         object
endStationId        object
endStationName      object
peekAlloc           object
regionName          object
routeId             object
routeName           object
routeTypeCd         object
routeTypeName       object
startMobileNo       object
startStationId      object
startStationName    object
upFirstTime         object
upLastTime          object
nPeekAlloc          object
dtype: object

In [112]:
colList = inputDataFrame.columns
for col in range(len(colList)):
    inputDataFrame[colList[col]].to_string()

In [232]:
inputDataFrame["companyId"] = pd.to_numeric(inputDataFrame["companyId"], )
# inputDataFrame["companyName"] = pd.to_numeric(inputDataFrame["companyName"])
# inputDataFrame["companyTel"] = pd.to_numeric(inputDataFrame["companyTel"])
inputDataFrame["districtCd"] = pd.to_numeric(inputDataFrame["districtCd"])
# inputDataFrame["downFirstTime"] = pd.to_numeric(inputDataFrame["downFirstTime"])
# inputDataFrame["downLastTime"] = pd.to_numeric(inputDataFrame["downLastTime"])
inputDataFrame["endMobileNo"] = pd.to_numeric(inputDataFrame["endMobileNo"])
inputDataFrame["endStationId"] = pd.to_numeric(inputDataFrame["endStationId"])
# inputDataFrame["endStationName"] = pd.to_numeric(inputDataFrame["endStationName"])
inputDataFrame["peekAlloc"] = pd.to_numeric(inputDataFrame["peekAlloc"])
# inputDataFrame["regionName"] = pd.to_numeric(inputDataFrame["regionName"])
inputDataFrame["routeId"] = pd.to_numeric(inputDataFrame["routeId"])
# inputDataFrame["routeName"] = pd.to_numeric(inputDataFrame["routeName"])
inputDataFrame["routeTypeCd"] = pd.to_numeric(inputDataFrame["routeTypeCd"])
# inputDataFrame["routeTypeName"] = pd.to_numeric(inputDataFrame["routeTypeName"])
inputDataFrame["startMobileNo"] = pd.to_numeric(inputDataFrame["startMobileNo"])
inputDataFrame["startStationId"] = pd.to_numeric(inputDataFrame["startStationId"])
# inputDataFrame["startStationName"] = pd.to_numeric(inputDataFrame["startStationName"])
# inputDataFrame["upFirstTime"] = pd.to_numeric(inputDataFrame["upFirstTime"])
# inputDataFrame["upLastTime"] = pd.to_numeric(inputDataFrame["upLastTime"])
inputDataFrame["nPeekAlloc"] = pd.to_numeric(inputDataFrame["nPeekAlloc"])
inputDataFrame.dtypes

지역                   object
노선                   object
시점                   object
종점                   object
승차                    int64
하차                   object
전체                    int64
companyId             int64
companyName          object
companyTel           object
districtCd            int64
downFirstTime        object
downLastTime         object
endMobileNo         float64
endStationId          int64
endStationName       object
peekAlloc           float64
regionName           object
routeId               int64
routeName            object
routeTypeCd           int64
routeTypeName        object
startMobileNo       float64
startStationId        int64
startStationName     object
upFirstTime          object
upLastTime           object
nPeekAlloc            int64
dtype: object

In [262]:
inputDataFrame['nPeekAlloc'] = inputDataFrame['nPeekAlloc'].fillna(0)
inputDataFrame.isnull().sum()
inputDataFrame.to_csv('C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/BusInfo/FINAL.csv', encoding='utf-8-sig')

## 조건에 맞는 노선 선택하기

In [5]:
_df = pd.read_csv('C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/BusInfo/FINAL.csv')

In [9]:
seatBusroute = _df[_df['routeTypeCd'] == 11] # 직행좌석형시내버스 중에서
print(seatBusroute.shape[0]) # 직행좌석형시내버스: 159대 중에서
manyPeopleRoute = seatBusroute.sort_values(by=['전체'], ascending=False).copy() # 이용자가 많은 
# manyPeopleRoute
testDataFrame = manyPeopleRoute.iloc[:50].sort_values(by=['nPeekAlloc'], ascending=False) # 상위 50대의 노선중 배차간격이 긴 버스
display(testDataFrame['routeId'])

159


1716    236000176
1429    234000079
1112    234000069
1116    234000882
781     228000184
775     228000177
28      200000145
774     228000176
158     204000070
160     234000886
1081    233000136
1357    234000051
613     234000873
861     229000028
1713    207000070
1092    233000139
35      200000108
1543    234000309
77      200000112
94      200000205
1129    234000313
33      200000104
78      200000119
134     205000002
36      200000110
779     228000175
776     228000394
20      200000120
82      233000032
1456    234000878
1131    234000324
1108    234000011
864     229000061
566     218000007
135     204000057
68      200000115
601     234000050
572     218000010
683     222000090
1000    232000092
114     204000081
112     206000007
265     232000092
571     219000016
682     222000084
84      200000150
568     219000006
565     219000013
81      233000031
80      233000031
Name: routeId, dtype: int64

In [ ]:
236000176,
234000079,
234000069,
234000882,
228000184,
228000177,
200000145,
228000176,
204000070,
234000886,
233000136,
234000051,
234000873,
229000028,
207000070,
233000139,
200000108,
234000309,
200000112,
200000205,
234000313,
200000104,
200000119,
205000002,
200000110,
228000175,
228000394,
200000120,
233000032,
234000878,
234000324,
234000011,
229000061,
218000007,
204000057,
200000115,
234000050,
218000010,
222000090,
232000092,
204000081,
206000007,
232000092,
219000016,
222000084,
200000150,
219000006,
219000013,
233000031,
233000031,